In [5]:
import json

with open("./vendor-purchased-2019/vendor-purchased-2019_tweets.json", "r") as f:
    JSON = json.load(f)
print(len(JSON))

dataset = []
for data in JSON:
    instance = dict()
    features = dict()
    
    UTC = data["created_at"]
    user = data["user"]
    
    #Get "friends/follower_ratio" feature
    if user["followers_count"] == 0 :
        ratio = 0
    else:
        ratio = user["friends_count"]/user["followers_count"]
        
    features["friends/follower_ratio"] = ratio
    
    #Get "is_bot_in_name" feature
    if "bot" in user["screen_name"]:
        is_name = 1
    else:
        is_name = 0
        
    features["is_bot_in_name"] = is_name
    
    #Save ID and extracted features
    instance["id"] = user["id"]
    instance["features"] = features
    
    dataset.append(instance)
    
print(dataset[0])

1088
{'id': 53805021, 'features': {'friends/follower_ratio': 119.66666666666667, 'is_bot_in_name': 0}}


In [17]:
with open("./vendor-purchased-2019/vendor-purchased-2019.tsv", "r") as f :
    TSV = f.readlines()
print(len(TSV))    

labels = []
for label in TSV:
    #All user_ids in TSV are bot.
    bot_user_id = label.split()[0]
    labels.append(int(bot_user_id))

print(labels[0])

1088
736629545512632320


In [30]:
def divide_dataset(X, y) :
    from sklearn.model_selection import train_test_split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
    return (X_train, X_test, y_train, y_test)

#X is features, y is labels
X = []
Y = []
for data in dataset :
    #Add label False to Y if user in JSON exists in TSV
    if data["id"] in labels :
        Y.append(False)
    else:
        continue 
        
    #Add every features to X.
    features = []
    for feature in data["features"].values():
        features.append(feature)
        
    X.append(features)
print(X[0], Y[0])
print(len(X), len(Y))

X_train, X_test, y_train, y_test = divide_dataset(X, Y)

[119.66666666666667, 0] False
1087 1087


In [31]:
import time
import datetime
def train(classifier, name, param_grid=None) :
    start_time = time.time()
    if param_grid == None :
        classifier.fit(X_train, y_train)
        results[name] = dict(model=classifier)
    else :
        grid = GridSearchCV(classifier, param_grid, cv=10, scoring='accuracy', n_jobs=2) # Do a 10-fold cross validation
        grid.fit(X, y) # fit the grid with data
        results[name] = dict(grid=grid, model=classifier)
    #total_time = datetime.datetime.fromtimestamp(time.time() - start_time)
    total_time = datetime.timedelta(seconds=time.time() - start_time)
    print("Training time : " + str(total_time))#.strftime('%H:%M:%S'))